<a href="https://colab.research.google.com/github/mohammedterry/NLP_for_ML/blob/master/NLG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2

In [0]:
!git clone https://github.com/openai/gpt-2
import os
os.chdir('gpt-2')
!pip3 install -r requirements.txt
!python3 download_model.py 345M

Cloning into 'gpt-2'...
remote: Enumerating objects: 174, done.
remote: Total 174 (delta 0), reused 0 (delta 0), pack-reused 174
Receiving objects: 100% (174/174), 4.35 MiB | 8.41 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Fetching checkpoint: 1.00kit [00:00, 654kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 48.4Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 647kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:29, 48.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 5.64Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 41.9Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 39.1Mit/s]                                                       


In [0]:
!python3 src/interactive_conditional_samples.py --model_name=345M --nsamples=3 --length=30 --top_k 40 --temperature 0.7

2019-05-14 00:23:41.181344: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-14 00:23:41.181822: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2f5d080 executing computations on platform Host. Devices:
2019-05-14 00:23:41.181877: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2019-05-14 00:23:51.043804: W tensorflow/core/framework/allocator.cc:124] Allocation of 205852672 exceeds 10% of system memory.
Model prompt >>> curiouser and curiouser
======================================== SAMPLE 1 ========================================
, the more I'm sure it's a good thing. That's because the problem with the

# Training Data

In [4]:
import nltk
nltk.download('punkt')
nltk.download('gutenberg')
from nltk.corpus import gutenberg
sentences = gutenberg.sents('carroll-alice.txt') #'shakespeare-macbeth.txt') #

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [5]:
text = '\n'.join([' '.join(sentence) for sentence in sentences])
print(text[100:500])

e was beginning to get very tired of sitting by her sister on the bank , and of having nothing to do : once or twice she had peeped into the book her sister was reading , but it had no pictures or conversations in it , ' and what is the use of a book ,' thought Alice ' without pictures or conversation ?'
So she was considering in her own mind ( as well as she could , for the hot day made her feel 


#  NGrams

In [0]:
import numpy as np
from collections import Counter
from difflib import SequenceMatcher 


class NgramModel():
  def __init__(self,ngram_size,chars=False):
    self.ngram_size = ngram_size
    self.chars=chars
    
  def fit(self,text):
    ngram_counter = Counter(self._ngrams(text))
    self.ngram_counter = {ngram: count for ngram, count in zip(ngram_counter, self._normalise(ngram_counter.values()))}
              
  def generate(self, words, iterations=30, k_beams = 3, diversity_threshold = .9):
    beams = [(1., words, self._ngrams(words)[-1])]
    for _ in range(iterations):
      #first we score next for each beam (based on frequency & similarity of ngrams) and keep the best k
      candidates = []
      for p, ws, prev_ngram in beams:
        prev_ngram = ' '.join(prev_ngram.split()[1:])
        new_candidates = [(p + f + self._compare(prev_ngram, ' '.join(ngram.split()[:-1]) ) , ws + [' ',''][self.chars] + ngram.split()[-1], ngram ) for ngram,f in self.ngram_counter.items() ] 
        new_candidates = sorted(new_candidates,reverse=True)[:k_beams]                
        candidates.extend(new_candidates)
      #now we filter out any beams that are too similar to other beams (to keep diversity)
      candidates = sorted(candidates,reverse=True)
      beams = [candidates[0]]
      for p,ws,ng in candidates[1:]:
        similarity = max([self._compare(ws,ws2) for _,ws2,_ in beams])
        if similarity > diversity_threshold:
          p = 0
        beams.append((p,ws,ng))
      #now we keep only the top k beams (for memory efficiency)
      beams = sorted(beams, reverse=True)[:k_beams]
    return [ws.replace('_',' ') for _,ws,_ in beams]
       
  def _tokenise(self,text):
    return ''.join([c if ord('a') <= ord(c) <= ord('z') else f' {c} ' for c in text.lower()]).split()
  
  def _ngrams(self,text):
      tokens = self._tokenise(text)
      if self.chars: #char-level ngrams
        tokens = list(' '.join(tokens).replace(' ','_'))
      ngrams = zip(*[tokens[i:] for i in range(self.ngram_size)])
      return [" ".join(ngram) for ngram in ngrams]
  
  def _normalise(self, X):
    s = sum(X)
    return [x/s for x in X]
  
  def _compare(self, string1,string2):
    return SequenceMatcher(None,string1, string2).ratio()

In [0]:
ng = NgramModel(4)
ng.fit(text)
ng.generate("i like to eat Chinese food")

["i like to eat Chinese food out a new kind of rule , ' and that ' s the most important piece of evidence we ' ve no time to wash the things between whiles .",
 "i like to eat Chinese food these cakes , ' she said to herself , ' i ' m not a serpent , i tell you ! ' said the king . ' when we were",
 "i like to eat Chinese food these cakes , ' she thought , ' till tomorrow - - ' ' i ' m not a serpent , i tell you ! ' said the king ."]

In [0]:
ng = NgramModel(4, chars=True)
ng.fit(text)
ng.generate("i like to eat Chinese food")

["i like to eat Chinese food down a little , ' said the wa",
 "i like to eat Chinese fooderself , ' said the was she wa",
 "i like to eat Chinese fooderseshought alice , ' said the"]

# LSTM

In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!ls

--2019-05-21 16:44:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-05-21 16:44:49--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-05-21 16:44:49--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
import numpy as np 
import keras

class LSTMmodel():  
  def fit(self,text,epochs=30, wordvec_filepath = 'glove.6B.100d.txt', wordvec_dim = 100):
    self._load(text)
    self._build(wordvec_filepath, wordvec_dim)
    self._train(epochs)

  def generate(self,words,i=30):
    for _ in range(i):
      x = [self.token_idx[token] if token in self.token_idx else 1 for token in self._tokenise(words)] 
      x = keras.preprocessing.sequence.pad_sequences([x], maxlen=self.x_dim, padding = 'pre')
      y_hat = self.model.predict_classes(x, verbose=0)[0] #maximise
      words += ' ' + self.idx_token[y_hat]
    return words
  
  def _tokenise(self,text):
    return ''.join([c if ord('a') <= ord(c) <= ord('z') else f' {c} ' for c in text.lower()]).split()
    
  def _load(self, text):    
    self.idx_token = dict(enumerate(set(self._tokenise(text)),start=2))
    self.idx_token[0] = '<PAD>'
    self.idx_token[1] = '<UNK>' 
    self.token_idx = {word:i for i,word in self.idx_token.items()}       
    token_ids = [[self.token_idx[token] for token in self._tokenise(sentence)] for sentence in text.split('\n')]
    inouts = [tokens[:i+1] for tokens in token_ids for i in range(1,len(tokens))]
    self.x_dim = max([len(x) for x in inouts]) - 1
    self.y_dim = len(self.idx_token) 
    inouts = np.array(keras.preprocessing.sequence.pad_sequences(inouts,maxlen=self.x_dim + 1, padding='pre'))
    self.X, self.Y = inouts[:,:-1], inouts[:,-1]
    
  def _build(self,filepath,wv_dim):
    embeddings = self._embeddingmatrix(self._wordvectors(filepath), wv_dim)
    self.model = keras.models.Sequential()
    self.model.add(keras.layers.Embedding(self.y_dim, wv_dim, input_length=self.x_dim, embeddings_initializer=keras.initializers.Constant(embeddings) , trainable=False))
    self.model.add(keras.layers.LSTM(200, return_sequences = True))
    self.model.add(keras.layers.LSTM(200))
    self.model.add(keras.layers.Dense(self.y_dim, activation='softmax'))
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
  def _train(self,epochs):
    earlystop =  keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=0, mode='auto')
    onehot_y = keras.utils.to_categorical(self.Y, num_classes=self.y_dim)
    self.model.fit(self.X, onehot_y, epochs=epochs, verbose=1, callbacks=[earlystop])  
  
  def _wordvectors(self,file_path):
    vectors = {}
    with open(file_path) as f:
      for line in f.readlines():
        values = line.split()
        vectors[values[0]] = np.array(values[1:],dtype='float32')
    return vectors
  
  def _embeddingmatrix(self, wordvectors, wv_dim):
    matrix = np.zeros((self.y_dim, wv_dim))
    for word,idx in self.token_idx.items():
      if word.lower() in wordvectors:
        matrix[idx] = wordvectors[word.lower()]
    return matrix

In [19]:
lstm = LSTMmodel()
lstm.fit(text[1000:10000])

Epoch 1/30
1991/1991 [==============================] - 28s 14ms/step - loss: 5.6688 - acc: 0.0638
Epoch 2/30


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1991/1991 [==============================] - 27s 14ms/step - loss: 5.3495 - acc: 0.0713
Epoch 3/30
1991/1991 [==============================] - 27s 13ms/step - loss: 5.2858 - acc: 0.0723
Epoch 4/30
1991/1991 [==============================] - 26s 13ms/step - loss: 5.1999 - acc: 0.0784
Epoch 5/30
1991/1991 [==============================] - 28s 14ms/step - loss: 5.0569 - acc: 0.0884
Epoch 6/30
1991/1991 [==============================] - 26s 13ms/step - loss: 4.8775 - acc: 0.1150
Epoch 7/30
1991/1991 [==============================] - 26s 13ms/step - loss: 4.6495 - acc: 0.1281
Epoch 8/30
1991/1991 [==============================] - 27s 14ms/step - loss: 4.4180 - acc: 0.1497
Epoch 9/30
1991/1991 [==============================] - 26s 13ms/step - loss: 4.1730 - acc: 0.1577
Epoch 10/30
1991/1991 [==============================] - 26s 13ms/step - loss: 3.9347 - acc: 0.1768
Epoch 11/30
1991/1991 [==============================] - 27s 13ms/step - loss: 3.7110 - acc: 0.2004
Epoch 12/30
1991/19

In [21]:
lstm.generate("Alice was")

'Alice was not a bit hurt , and she jumped up on to her feet in a moment : she looked up , but it was all dark overhead ; and the'

## ULMFIT (AWD-LSTM & Transfer Learning)

In [0]:
training_data = [sentence.split()[:i+1] for sentence in text.split('\n') for i in range(1,len(sentence.split()))]
import pandas as pd
df = pd.DataFrame({
    "text":[' '.join(sample[:-1]) for sample in training_data],
    "label":[sample[-1] for sample in training_data],
})
df.head()

,text,label
0,I,'
1,I ',m
2,I ' m,sure
3,I ' m sure,_I_
4,I ' m sure _I_,shan


In [0]:
from fastai.text import language_model_learner, TextLMDataBunch,untar_data,URLs,AWD_LSTM
ulmfit = language_model_learner(TextLMDataBunch.from_df(untar_data(URLs.IMDB_SAMPLE),train_df = df , valid_df = df ), AWD_LSTM, drop_mult=.5)

In [0]:
ulmfit.predict(seed, n_words=100)

'..how was a large - and - large , - off , - Mouse : a ) being over - used when it was found out . The Use Was Sudden That ( a ) was a French by - hands , used to be used for the Rome Times and French Times . In " The Great Can See , " the four - used Through The Door of the Being was a French Just'

In [0]:
ulmfit.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.214114,6.888145,0.014286,00:09


In [0]:
ulmfit.predict(seed, n_words=100)

'..how was in Paris when " There was a French - French No - Get Me ! right there to see ! " , he said , " when i was in the back of the Sea of , i \' m You i \' ve Your Eyes " . The French and English Times They Are a Of The Times , The Times \' Talk OF The Times ,'

# Char-Level LSTM

In [0]:
import numpy as np 
import keras

class charLSTMmodel():
  
  def fit(self,text,epochs=100):
    self._load(text)
    self._build()
    self._train(epochs)

  def generate(self,words,i=150):
    for _ in range(i):
      x = [self.token_idx[token] if token in self.token_idx else 1 for token in self._tokenise(words)] 
      x = keras.preprocessing.sequence.pad_sequences([x], maxlen=self.x_dim, padding = 'pre')
      y_hat = self.model.predict_classes(x, verbose=0)[0] #maximise
      words += self.idx_token[y_hat]
    return words.replace("_"," ")
  
  def _chunk(self,text,chunk_size = 100):
    return ''.join([c + '<S>' if not i % chunk_size else c for i,c in enumerate(text,start=1)]).split('<S>')
  
  def _tokenise(self,text):
    return list(' '.join(text.split()).replace(" ","_"))
    
  def _load(self, text):
    self.idx_token = dict(enumerate(set(self._tokenise(text)),start=2))
    self.idx_token[0] = '<PAD>'
    self.idx_token[1] = '<UNK>' 
    self.token_idx = {word:i for i,word in self.idx_token.items()}       
    token_ids = [[self.token_idx[token] for token in self._tokenise(sentence)] for sentence in self._chunk(text)]
    inouts = [tokens[:i+1] for tokens in token_ids for i in range(1,len(tokens))]
    self.x_dim = max([len(x) for x in inouts]) - 1
    self.y_dim = len(self.idx_token) 
    inouts = np.array(keras.preprocessing.sequence.pad_sequences(inouts,maxlen=self.x_dim + 1, padding='pre'))
    self.X, self.Y = inouts[:,:-1], inouts[:,-1]
    
  def _build(self):
    self.model = keras.models.Sequential()
    self.model.add(keras.layers.Embedding(self.y_dim, 10, input_length=self.x_dim))
    self.model.add(keras.layers.LSTM(150, return_sequences = True))
    self.model.add(keras.layers.LSTM(100))
    self.model.add(keras.layers.Dense(self.y_dim, activation='softmax'))
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
  def _train(self,epochs):
    earlystop =  keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    onehot_y = keras.utils.to_categorical(self.Y, num_classes=self.y_dim)
    self.model.fit(self.X, onehot_y, epochs=epochs, verbose=1, callbacks=[earlystop])  

In [0]:
clstm = charLSTMmodel()
clstm.fit(text, epochs=50)

Epoch 1/50
9379/9379 [==============================] - 107s 11ms/step - loss: 3.0769 - acc: 0.2306
Epoch 2/50


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


9379/9379 [==============================] - 105s 11ms/step - loss: 2.9091 - acc: 0.2452
Epoch 3/50
9379/9379 [==============================] - 105s 11ms/step - loss: 2.6316 - acc: 0.2919
Epoch 4/50
9379/9379 [==============================] - 104s 11ms/step - loss: 2.4382 - acc: 0.3276
Epoch 5/50
9379/9379 [==============================] - 105s 11ms/step - loss: 2.7868 - acc: 0.2730
Epoch 6/50
9379/9379 [==============================] - 104s 11ms/step - loss: 2.4019 - acc: 0.3371
Epoch 7/50
9379/9379 [==============================] - 104s 11ms/step - loss: 2.2483 - acc: 0.3688
Epoch 8/50
9379/9379 [==============================] - 105s 11ms/step - loss: 2.1832 - acc: 0.3847
Epoch 9/50
9379/9379 [==============================] - 105s 11ms/step - loss: 2.1257 - acc: 0.3983
Epoch 10/50
9379/9379 [==============================] - 105s 11ms/step - loss: 2.0774 - acc: 0.4100
Epoch 11/50
9379/9379 [==============================] - 105s 11ms/step - loss: 2.0267 - acc: 0.4235
Epoch 12/

In [0]:
clstm.generate("indubitably ")

indubitably elice , and her eyes filled with tears again . ' You ought to be ashamed of yourself ,' said Alice , ' to speak to this mouse , she was tomes on one h
indubitably elice , and her eyes firled with tee whonged I wuber , spe bnowg a warden of yoor again . ' You ought to mriwpy and on the gool of any one , so gone ,


# char-LSTM with Attention & Transfer Learning

In [0]:
from textgenrnn import textgenrnn

clstm_att = textgenrnn()
clstm_att.generate()

[PS4] LF4M BOOSALIQ a promote of my computer at the mouse of the users made it to the stranger and then the shopping and all the best for the teammates. Thanks for a set of some possible shots at all time his boys on the entire popular decade and she didn't see an increase it would be transferent,



In [0]:
clstm_att.train_on_texts(text.split('\n'),num_epochs=10)

Training on 9,522 character sequences.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
74/74 [==============================] - 29s 387ms/step - loss: 1.7788
####################
Temperature: 0.2
####################
' I had so the she she she she she was not to one of the she she she she she she she she she she was sure the here !'

' S sure the she she she she she was surcement to she was sure to she she she she she she she was now in the sure the sure to when the she she she she she she she she was surportility to the she she she she she she she she she she she she was not the sure in the sure in the sure the sure it '  she

' Alice '  ' she she she she she she she she she was not such a sure the she she she she she she she was now to the she she she she she she she she she she was not such a little she was now to such she was not such a little surcemen than the how her she was heards , and she was surced to such she 

####################
Te

In [0]:
clstm_att.generate()

The Mouse was not as the time she was to one of estist if I ' more , and began to find me , I ' ll male at all the try she stops down the same again .

